<a href="https://colab.research.google.com/github/Marvin2798/CNN-Project-/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Link : https://github.com/ironhack-labs/project-1-deep-learning-image-classification-with-cnn/blob/main/README.md#project-i--deep-learning-image-classification-with-cnn

Loading and Data Processing

In [ ]:
from tensorflow.keras.datasets import cifar10
#loading the dataset since we can access it through keras
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
#normalise the pixel values to 0 and 1 by dividing by 255
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_train
X_test


array([[[[9.52876417e-06, 6.75456658e-06, 2.95512291e-06],
         [9.58907276e-06, 6.69425799e-06, 2.83450549e-06],
         [9.95092432e-06, 6.99580096e-06, 3.07574010e-06],
         ...,
         [8.26228279e-06, 5.72931958e-06, 2.17111074e-06],
         [7.59888735e-06, 5.48808521e-06, 2.17111074e-06],
         [6.99580096e-06, 5.12623365e-06, 1.99018496e-06]],

        [[9.16691260e-06, 6.75456658e-06, 3.07574010e-06],
         [9.10660401e-06, 6.63394940e-06, 2.41234534e-06],
         [9.58907276e-06, 6.87518377e-06, 2.71388831e-06],
         ...,
         [8.20197420e-06, 5.72931958e-06, 1.86956754e-06],
         [7.53857876e-06, 5.48808521e-06, 1.92987636e-06],
         [7.17672719e-06, 5.30715943e-06, 2.05049355e-06]],

        [[9.10660401e-06, 6.63394940e-06, 2.83450549e-06],
         [9.10660401e-06, 6.57364080e-06, 1.99018496e-06],
         [9.52876417e-06, 6.69425799e-06, 2.17111074e-06],
         ...,
         [8.38289998e-06, 5.91024582e-06, 2.05049355e-06],
         [

In [ ]:
#test


Model Architecture

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define the model
model = Sequential()

# Add Input layer explicitly
model.add(Input(shape=(32, 32, 3)))

# Add convolutional layers and pooling layers
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add fully connected (dense) layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout to avoid overfitting
model.add(Dense(10, activation='softmax'))  # Output layer (10 classes)


In [7]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
# Display the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,202 (625.79 KB)

 Trainable params: 160,202 (625.79 KB)

 Non-trainable params: 0 (0.00 B)

Model Evaluation